In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
stocks = pd.read_csv('data_challenge_stock_prices.csv')
index = pd.read_csv('data_challenge_index_prices.csv')

In [3]:
def getBPS(df):
    mat = df.to_numpy()
    returns = (mat[1:] - mat[:len(mat)-1])/mat[:len(mat)-1]
    return returns * 10000

In [4]:
index.shape

(200000, 15)

In [5]:
st = getBPS(stocks)
indices = getBPS(index)

In [6]:
st.shape

(199999, 100)

In [7]:
from scipy import stats

In [8]:
corr = np.zeros((100,100))

In [9]:
for i in range(100):
    for j in range(100):
        corr[i][j] = stats.pearsonr(st[:,i],st[:,j])[0]

In [10]:
maxcorr = np.zeros(100,dtype=np.int32)
for i in range(100):
    ind = (i+1)%100
    for j in range(100):
        if i==j:
            continue
        if corr[i][j]>corr[i][ind]:
            ind = j
    maxcorr[i] = ind

In [11]:
maxcorr

array([88, 90, 98, 98, 88, 97, 98, 98, 99, 98, 90, 88, 99, 97, 88, 97, 98,
       98, 88, 98, 88, 88, 88, 98, 97, 88, 88, 99, 98, 99, 99, 97, 98, 98,
       97, 97, 99, 97, 88, 88, 99, 99, 98, 88, 99, 88, 99, 98, 97, 88, 99,
       98, 99, 98, 88, 99, 97, 99, 98, 99, 98, 97, 97, 98, 97, 97, 98, 88,
       97, 99, 97, 99, 98, 99, 99, 88, 97, 99, 88, 88, 99, 98, 88, 97, 99,
       97, 97, 98, 90, 97, 88, 98, 99, 97, 97, 97, 99, 95, 91, 96])

In [12]:
buckets = {i:[] for i in maxcorr}

In [13]:
for i in range(100):
    buckets[maxcorr[i]].append(i)

In [14]:
for i in buckets:
    print(i,buckets[i])

88 [0, 4, 11, 14, 18, 20, 21, 22, 25, 26, 38, 39, 43, 45, 49, 54, 67, 75, 78, 79, 82, 90]
90 [1, 10, 88]
98 [2, 3, 6, 7, 9, 16, 17, 19, 23, 28, 32, 33, 42, 47, 51, 53, 58, 60, 63, 66, 72, 81, 87, 91]
97 [5, 13, 15, 24, 31, 34, 35, 37, 48, 56, 61, 62, 64, 65, 68, 70, 76, 83, 85, 86, 89, 93, 94, 95]
99 [8, 12, 27, 29, 30, 36, 40, 41, 44, 46, 50, 52, 55, 57, 59, 69, 71, 73, 74, 77, 80, 84, 92, 96]
95 [97]
91 [98]
96 [99]


In [15]:
buckets[88].extend(buckets[90])
buckets.pop(90)
buckets.pop(95)
buckets.pop(91)
buckets.pop(96)
buckets[98].append(98)
buckets[97].append(97)
buckets[99].append(99)

In [16]:
sectors = []
for i in buckets:
    sectors.append(buckets[i])
sectors = np.array(sectors)
sectors.shape

(4, 25)

In [17]:
np.save("sectors.npy",sectors)

In [18]:
def correlation(vec1, vec2):
    dim1 = vec1.shape[1]
    dim2 = vec2.shape[1]
    correl = np.zeros((dim1,dim2),dtype=np.float32)
    for i in range(dim1):
        for j in range(dim2):
            correl[i][j] = stats.pearsonr(vec1[:,i],vec2[:,j])[0]
    return correl

In [19]:
import seaborn as sbn

In [20]:
stock_sectorwise = np.array([st[:,sectors[i]] for i in range(4)])

In [21]:
import matplotlib.pyplot as plt

In [22]:
# for i in range(4):
#     for j in range(i,4):
#         plt.title(f"Sector {i} vs Sector {j}")
#         sbn.heatmap(correlation(stock_sectorwise[i],stock_sectorwise[j]),vmin=0,vmax=0.5)
#         plt.show()

In [23]:
from sklearn.linear_model import LinearRegression

In [24]:
def test(Func):   
    pred_corr = np.zeros((15,4))
    for i in range(4):
        for j in range(15):
            x = stock_sectorwise[i]     # 199999,25
            y = indices                 # 199999,1
            x_tr = x[0:12000]
            x_te = x[12000:]
            y_tr = y[0:12000,j]
            y_te = y[12000:,j]
            model = Func()
            model.fit(x_tr,y_tr)
            y_pred = model.predict(x_te)
            pecorr = stats.pearsonr(y_te[:],y_pred[:])[0]
            pred_corr[j][i] = pecorr
    
    posn = np.argmax(pred_corr,axis=1)
    for i in range(15):
        print(i,pred_corr[i],posn[i])
    return pred_corr


In [25]:
from sklearn.neural_network import MLPRegressor

In [26]:
print("Linear")
pc = test(LinearRegression)

Linear
0 [0.44428666 0.08731293 0.08200398 0.08465401] 0
1 [0.05599844 0.05183528 0.31142448 0.05264355] 2
2 [0.20657407 0.01151913 0.01726661 0.01918735] 0
3 [0.0298753  0.34350168 0.02661995 0.02855217] 1
4 [0.09505984 0.07339035 0.08738046 0.44746208] 3
5 [0.01818233 0.0265306  0.21575747 0.0275874 ] 2
6 [0.08724768 0.44362346 0.09761592 0.08195443] 1
7 [0.30144975 0.05163767 0.05164508 0.05090786] 0
8 [0.04894639 0.30381171 0.05049344 0.05297393] 1
9 [0.02944939 0.02110999 0.03250083 0.34517321] 3
10 [0.01015871 0.02096384 0.3408217  0.01171519] 2
11 [-0.00015145  0.00158011 -0.00180775 -0.00240251] 1
12 [ 0.00549766 -0.00246269  0.00126297 -0.00114666] 0
13 [-0.00341212  0.00479849  0.00124561 -0.00108384] 1
14 [0.01209144 0.00158175 0.00911624 0.20853675] 3


In [27]:
# For all except 11,12 and 13, we have figured out which sector.

In [28]:
# print("MLP")
# import warnings
# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     pred_corr = test(MLPRegressor)

In [29]:
# A simple neural net gave reasonable correlation for all indices including 11,12 and 13. 

In [26]:
index_sector = {0:0,1:2,2:0,3:1,4:3,5:2,6:1,7:0,8:1,9:3,10:2,11:0,12:3,13:1,14:3}

In [46]:
def train_and_evaluate(ind_num,sector_num,train_len,test_len,model, do_scale=False, do_poly=-1):
    train_x = stock_sectorwise[sector_num][:train_len]
    train_y = indices[:train_len,ind_num]
    test_x = stock_sectorwise[sector_num][train_len:train_len+test_len]
    test_y = indices[train_len:train_len+test_len,ind_num]

    print(train_y.shape, test_y.shape)

    if do_scale:
        # scale data
        scaler = sklearn.preprocessing.MinMaxScaler().fit(train_x)
        train_x = scaler.transform(train_x)
        test_x = scaler.transform(test_x)

    if do_poly != -1:
        poly = sklearn.preprocessing.PolynomialFeatures(degree=do_poly)
        train_x = poly.fit_transform(train_x)
        test_x = poly.fit_transform(test_x)

    model.fit(train_x,train_y)
    y_pred = model.predict(test_x)
    pecorr = stats.pearsonr(test_y,y_pred)[0]
    print(f"Predictive correlation for index {ind_num} : {pecorr}")
    return model

In [50]:
split_ratio = 0.8
train_len = int(indices.shape[0]*0.8)
test_len = indices.shape[0] - train_len

In [51]:
#Index 0:
#Linear Model
ind_num = 0
sector_num = index_sector[ind_num]
mod0 = LinearRegression()
# mod0 = sklearn.linear_model.Ridge()
# mod0 = sklearn.svm.SVR()      # quadratic time
mod0 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod0)

(159999,) (40000,)
Predictive correlation for index 0 : 0.44672797701419487


In [52]:
#Index 1:
#Neural Net
ind_num = 1
sector_num = index_sector[ind_num]
mod1 = MLPRegressor(hidden_layer_sizes=(100))
# mod1 = LinearRegression()
mod1 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod1, do_scale=True)

# [MLP 100] scaling gave a 1% increase
# Linear+poly-2: 37%

(159999,) (40000,)
Predictive correlation for index 1 : 0.41321039706005475


In [53]:
#Index 2:
#Neural Net
ind_num = 2
sector_num = index_sector[ind_num]
mod2 = MLPRegressor(hidden_layer_sizes=(20))
mod2 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod2)

(159999,) (40000,)
Predictive correlation for index 2 : 0.5876291077266979


In [54]:
#Index 3:
#Neural Net
ind_num = 3
sector_num = index_sector[ind_num]
mod3 = MLPRegressor(hidden_layer_sizes=(5))
mod3 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod3)

(159999,) (40000,)
Predictive correlation for index 3 : 0.43966585997027485


In [55]:
#Index 4:
#Linear Model
ind_num = 4
sector_num = index_sector[ind_num]
# mod4 = LinearRegression()
mod4 = MLPRegressor(hidden_layer_sizes=20)
mod4 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod4)

(159999,) (40000,)
Predictive correlation for index 4 : 0.44592349898499234


In [56]:
#Index 5:
#Neural Net
ind_num = 5
sector_num = index_sector[ind_num]
mod5 = MLPRegressor(hidden_layer_sizes=(10))
mod5 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod5)

(159999,) (40000,)
Predictive correlation for index 5 : 0.5607516425359469


In [57]:
#Index 6:
#Neural Net
ind_num = 6
sector_num = index_sector[ind_num]
mod6 = MLPRegressor(hidden_layer_sizes=(5))
# mod6 = LinearRegression()           # almost the same as MLP-5
mod6 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod6)

(159999,) (40000,)
Predictive correlation for index 6 : 0.44597594692202813


In [58]:
#Index 7:
#Neural Net
ind_num = 7
sector_num = index_sector[ind_num]
mod7 = MLPRegressor(hidden_layer_sizes=(80))
# mod7 = LinearRegression()     # usual:30%, scale+deg-2:36%
mod7 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod7)

# lin-reg: 30%
# lin + [scale] + deg-2: 36%
# MLP-20: 36%
# MLP-80: 40%

(159999,) (40000,)
Predictive correlation for index 7 : 0.4044309474631219


In [59]:
#Index 8:
#Neural Net
ind_num = 8
sector_num = index_sector[ind_num]
mod8 = MLPRegressor(hidden_layer_sizes=(80))
mod8 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod8)

(159999,) (40000,)
Predictive correlation for index 8 : 0.41417626053650775


In [60]:
#Index 9:
#Neural Net
ind_num = 9
sector_num = index_sector[ind_num]
mod9 = MLPRegressor(hidden_layer_sizes=(80))
mod9 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod9)

(159999,) (40000,)
Predictive correlation for index 9 : 0.43067265570020946


In [61]:
#Index 10:
#Neural Net
ind_num = 10
sector_num = index_sector[ind_num]
mod10 = MLPRegressor(hidden_layer_sizes=(80))
mod10 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod10)

(159999,) (40000,)
Predictive correlation for index 10 : 0.42557732062066533


In [62]:
#Index 11:
#Neural Net
ind_num = 11
sector_num = index_sector[ind_num]
mod11 = MLPRegressor(hidden_layer_sizes=(80))
mod11 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod11)

(159999,) (40000,)
Predictive correlation for index 11 : 0.4342854580456461


In [63]:
#Index 12:
#Neural Net
ind_num = 12
sector_num = index_sector[ind_num]
mod12 = MLPRegressor(hidden_layer_sizes=(80))
mod12 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod12)

(159999,) (40000,)
Predictive correlation for index 12 : 0.4252763460434209


In [64]:
#Index 13:
#Neural Net
ind_num = 13
sector_num = index_sector[ind_num]
mod13 = MLPRegressor(hidden_layer_sizes=(80))
mod13 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod13)

(159999,) (40000,)
Predictive correlation for index 13 : 0.43230049959412215


In [65]:
#Index 14:
#Neural Net
ind_num = 14
sector_num = index_sector[ind_num]
mod14 = MLPRegressor(hidden_layer_sizes=(20))
mod14 = train_and_evaluate(ind_num,sector_num,train_len,test_len,mod14)

(159999,) (40000,)
Predictive correlation for index 14 : 0.5779839184017699


In [66]:
models = [mod0,mod1,mod2,mod3,mod4,mod5,mod6,mod7,mod8,mod9,mod10,mod11,mod12,mod13,mod14]

In [72]:
import joblib

for i,mod in enumerate(models):
    joblib.dump(mod, f'models/{i}.pkl')

In [67]:
def predict(returns): #Takes stock returns as input, returns index return predictions.
    res = []
    for i in range(15):
        inp = returns[:,sectors[index_sector[i]]]
        res.append(models[i].predict(inp))
    
    return np.array(res).T

In [82]:
predictions = predict(st[160000:])
covmat = ((predictions.T @ predictions)/len(predictions)) - np.outer(np.mean(predictions,axis=0),np.mean(predictions,axis=0))

In [85]:
covmat.diagonal()

array([1.55232620e+00, 2.50601401e+03, 4.51278900e-01, 2.46218576e+00,
       1.51903318e+00, 4.00497420e-01, 1.51050232e+00, 1.19767012e-01,
       1.19582003e-01, 2.50721004e+00, 2.50653915e+00, 7.63726201e-01,
       7.76832704e-01, 7.62416024e-01, 4.15769606e-01])

In [132]:
indices_to_trade = list(range(15))

In [133]:
timesteps = 40000
basev = 160000-1
stock_prices = st[basev:basev+timesteps]
pred = predict(stock_prices)[:,indices_to_trade]
cov_submat = covmat[indices_to_trade,:][:,indices_to_trade]

In [137]:
def tradingstrategy(timesteps, pred, cov_submat, indices_to_trade):
  # To return a portfolio of length timesteps, containing the position of each index we're trading in at time T

  allocs = np.zeros((timesteps,len(indices_to_trade)))
  pos_1_worked = 0

  for i in range(timesteps):
    alloc = np.zeros(len(indices_to_trade))
    ret_ratios = pred[i]/np.sqrt(cov_submat.diagonal())

    # try +1 to largest
    pos_1_works = True

    if np.max(ret_ratios) <= 0:
      continue

    arg_max_ret = np.argmax(ret_ratios)
    alloc[arg_max_ret] = 1
    sum_pos = 1

    for j in range(len(alloc)):
      if alloc[j]!=1 and ret_ratios[j]>0:
        alloc[j] = ret_ratios[j]/ret_ratios[arg_max_ret]
        sum_pos += alloc[j]
    
    sum_ret_neg = np.sum(ret_ratios[ret_ratios<0])

    for j in range(len(alloc)):
      if ret_ratios[j]<0:
        alloc[j] = -sum_pos * ret_ratios[j]/sum_ret_neg
        if alloc[j]<-1:
          pos_1_works = False
          break
    
    if pos_1_works:
      pos_1_worked += 1

    if not pos_1_works:
      # try -1 to smallest
      alloc = np.zeros(len(indices_to_trade))

      if np.min(ret_ratios) >= 0:
        continue

      arg_min_ret = np.argmin(ret_ratios)
      alloc[arg_min_ret] = -1
      sum_neg = -1

      for j in range(len(alloc)):
        if alloc[j]!=-1 and ret_ratios[j]<0:
          alloc[j] = -1.0 * ret_ratios[j]/ret_ratios[arg_min_ret]
          sum_neg += alloc[j]
      
      sum_ret_pos = np.sum(ret_ratios[ret_ratios>0])

      if i==0:
        print(arg_min_ret, sum_neg, sum_ret_pos)

      for j in range(len(alloc)):
        if ret_ratios[j]>0:
          alloc[j] = -sum_neg * ret_ratios[j]/sum_ret_pos
          if alloc[j]>1:
            print(alloc)
            alloc = np.zeros(len(indices_to_trade))
            print(ret_ratios, "none works", i, j)
            break
    
    allocs[i] = alloc

  print(pos_1_worked)
  
  return allocs

In [138]:
print(pred[0]/np.sqrt(cov_submat.diagonal()))

[ 1.50605219 -5.94029207  0.35278602 -0.78417665  0.8431377   3.13473069
  0.21869979  1.35551618  0.98358211 -0.13989541  0.231845    0.83115778
  0.44260166 -0.8270171   0.0534096 ]


In [139]:
portfolios = tradingstrategy(timesteps, pred, cov_submat, indices_to_trade)

1 -1.294781660436966 9.95351871144765
11308


In [140]:
print(portfolios[0])
print(np.sum(portfolios[0]))
print(portfolios[1])
print(np.sum(portfolios[1]))

[ 0.1959115  -1.          0.0458914  -0.13200978  0.10967772  0.40777457
  0.02844908  0.17632935  0.12794712 -0.02355026  0.03015905  0.10811934
  0.05757487 -0.13922162  0.00694767]
1.2836953722228372e-16
[-0.2404655  -0.78717    -0.07184016 -0.04553487 -0.00859168 -0.25339345
 -0.00208649 -0.211537    0.17427804 -0.1093394   1.          0.97868994
 -0.19990154 -0.19530748 -0.02780042]
4.85722573273506e-17


In [141]:
index_prices = indices[basev:basev+timesteps,indices_to_trade]

return_perc = []
for a,b in zip(portfolios,index_prices):
    return_perc.append(np.dot(a,b))

print(np.mean(return_perc),np.std(return_perc))

3.9853656125144923 3.4394867972190952
